<a href="https://colab.research.google.com/github/JinFree/yolov5/blob/master/Train_YOLOv5_with_Pascal_VOC_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 노트북은 Colab에서 YOLOv5s 모델을 VOC dataset으로 훈련하는 예제입니다.


PASCAL VOC Dataset을 다운로드 받은 후 압축을 풀어줍니다.

In [ ]:
!pip install gdown
!gdown 1w_WBizEt2e_u6T9iY-hkwA-fIVsJktbB
!tar -xf VOCtrainval_11-May-2012.tar
!rm VOCtrainval_11-May-2012.tar

실습하고 있는 데이터셋은 라벨링이 xml 파일 포맷으로 제공되며, 해당 포맷은 YOLOv5을 활용해 객체 인식 신경망을 훈련할 때 사용할 수 있는 파일 포맷이 아닙니다. 

convert2Yolo라고 하는 깃허브 저장소에서 Pascal VOC 데이터셋의 xml 파일 형태를 YOLOv5를 통한 훈련에 사용할 수 있도록 변환하는 기능을 제공합니다. 이를 활용해 xml 파일 포맷을 txt 파일 포맷으로 변환하겠습니다.

이 과정에 앞서 다음과 같이 Pascal VOC 데이터셋의 클래스 리스트가 있는 파일을 생성합니다.

In [ ]:
classes = ["aeroplane\n", "bicycle\n", "bird\n", "boat\n", "bottle\n", 
           "bus\n", "car\n", "cat\n", "chair\n", "cow\n", "diningtable\n", 
           "dog\n", "horse\n", "motorbike\n", "person\n", "pottedplant\n", 
           "sheep\n", "sofa\n", "train\n", "tvmonitor"]
with open("vocnames.txt", 'w') as f:
    f.writelines(classes)

vocnames.txt 파일이 생성된 것을 확인한 후 다음과 같이 VOCdevkit/VOC2012 폴더 아래에 labels 폴더를 생성한 후 convert2Yolo 저장소를 활용해 xml 파일을 txt 파일로 변환합니다

In [ ]:
!git clone https://github.com/ssaru/convert2Yolo.git
!cd convert2Yolo && python3 example.py --datasets VOC \
--img_path ../VOCdevkit/VOC2012/JPEGImages/ \
--label ../VOCdevkit/VOC2012/Annotations/ \
--convert_output_path ../VOCdevkit/VOC2012/JPEGImages/ \
--img_type ".jpg" \
--manifest_path ../ \
--cls_list_file ../vocnames.txt

다운로드 완료 후 훈련을 위한 이미지가 있는 폴더의 경로는 아래와 같습니다.

/content/VOCdevkit/VOC2012/JPEGImages/ 

YOLOv5를 통한 객체 인식 신경망을 훈련하기 위해 각 이미지 내 객체의 위치가 라벨링되어 있는 텍스트 파일은 이미지와 같은 폴더에 있습니다.

YOLOv5에서 훈련할 때 이미지와 같은 경로에 이미지와 동일한 이름의 라벨링 된 텍스트 파일이 있어야만 훈련을 수행할 수 있습니다. 구체적인 파일 구조는 아래와 같습니다.


```
$Object_Detection_Dataset/
                         ┗ 1.png
                         ┗ 1.txt
                         ┗ 2.jpg
                         ┗ 2.txt

```

이제 데이터를 훈련 데이터와 검증 데이터로 나누겠습니다.

In [ ]:
import os 
import shutil
from tqdm import tqdm
data_root = "/content/VOCData"
val_root = os.path.join(data_root, "val")
train_root = os.path.join(data_root, "train")
os.makedirs(val_root, exist_ok=True)
os.makedirs(train_root, exist_ok=True)

with open("/content/manifest.txt") as f:
    files = f.readlines()

for idx, img_path in tqdm(enumerate(files)):
    img_src = img_path.split('\n')[0]
    txt_src = os.path.splitext(img_src)[0] + ".txt"
    img_name = os.path.split(img_src)[-1]
    text_name = os.path.split(txt_src)[-1]
    if idx % 10 < 3:
        img_dst = os.path.join(val_root, img_name)
        text_dst = os.path.join(val_root, text_name)
    else:
        img_dst = os.path.join(train_root, img_name)
        text_dst = os.path.join(train_root, text_name)
    shutil.copy2(img_src, img_dst)
    shutil.copy2(txt_src, text_dst)

이제 YOLOv5를 VOC dataset으로 훈련하도록 하겠습니다.

학습을 위해 YOLOv5 환경을 구성하도록 하겠습니다.

In [ ]:
!git clone https://github.com/jetsonai/yolov5
%cd yolov5
%pip install -qr requirements.txt  # install

훈련에 앞서 YOLOv5로 추론을 할 수 있는 환경인지 테스트해봅니다.

검출은 아래와 같은 방식으로 할 수 있습니다.
```
python detect.py --source 0  # webcam
                          img.jpg  # image 
                          vid.mp4  # video
                          screen  # screenshot
                          path/  # directory
                         'path/*.jpg'  # glob
                         'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                         'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```

아래의 스크립트가 에러 없이 수행된다면 훈련할 준비가 되었습니다.

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images
import cv2
from google.colab.patches import cv2_imshow
cv2_imshow(cv2.imread('runs/detect/exp/zidane.jpg'))

이제 훈련을 위한 설정 파일을 준비합니다.

YOLOv5에는 기본적으로 VOC 데이터셋을 훈련하기 위한 VOC.yaml파일이 있습니다만, 저희는 별도로 yaml파일을 제작합니다.

내용은 아래와 같습니다.

YOLOv5는 darknet을 사용하는 이전의 YOLOv4와는 다르게 데이터셋이 있는 폴더 경로를 입력하여서 훈련에 사용할 수 있습니다.

```
train: 
  - /content/VOCData/train
val: 
  - /content/VOCData/val

# Classes
names:
  0: aeroplane
  1: bicycle
  2: bird
  3: boat
  4: bottle
  5: bus
  6: car
  7: cat
  8: chair
  9: cow
  10: diningtable
  11: dog
  12: horse
  13: motorbike
  14: person
  15: pottedplant
  16: sheep
  17: sofa
  18: train
  19: tvmonitor
```



In [ ]:
text_lines = '''
train:                          
  - /content/VOCData/train      
val:                            
  - /content/VOCData/val        
                                
# Classes                       
names:                          
  0: aeroplane                  
  1: bicycle                    
  2: bird                       
  3: boat                       
  4: bottle                     
  5: bus                        
  6: car                        
  7: cat                        
  8: chair                      
  9: cow                        
  10: diningtable               
  11: dog                       
  12: horse                     
  13: motorbike                 
  14: person                    
  15: pottedplant               
  16: sheep                     
  17: sofa                      
  18: train                     
  19: tvmonitor                 
'''
with open("/content/yolov5/vocdata.yaml", 'w') as f:
    f.write(text_lines)

이제 훈련을 수행하겠습니다.

In [ ]:
!python train.py --img 480 --batch 16 --epochs 20 --data vocdata.yaml --weights yolov5s.pt --cache

훈련이 완료되면 weight 파일을 다운로드합니다.

In [ ]:
from google.colab import files
files.download('/content/yolov5/runs/train/exp/weights/best.pt')